# IRIS 데이터 분류

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Input

In [2]:
!wget https://raw.githubusercontent.com/dhrim/cau_2022/master/material/deep_learning/iris.csv

--2022-01-19 07:43:56--  https://raw.githubusercontent.com/dhrim/cau_2022/master/material/deep_learning/iris.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2720 (2.7K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   2.66K  --.-KB/s    in 0s      

2022-01-19 07:43:56 (65.3 MB/s) - ‘iris.csv’ saved [2720/2720]



In [3]:
!ls -al
!head iris.csv

total 20
drwxr-xr-x 1 root root 4096 Jan 19 07:43 .
drwxr-xr-x 1 root root 4096 Jan 19 07:39 ..
drwxr-xr-x 4 root root 4096 Jan  7 14:33 .config
-rw-r--r-- 1 root root 2720 Jan 19 07:43 iris.csv
drwxr-xr-x 1 root root 4096 Jan  7 14:33 sample_data
septal_length,septal_width,petal_length,petal_width,setosa,versicolor,virginica
6.4,2.8,5.6,2.2,0,0,1
5.0,2.3,3.3,1.0,0,1,0
4.9,2.5,4.5,1.7,0,0,1
4.9,3.1,1.5,0.1,1,0,0
5.7,3.8,1.7,0.3,1,0,0
4.4,3.2,1.3,0.2,1,0,0
5.4,3.4,1.5,0.4,1,0,0
6.9,3.1,5.1,2.3,0,0,1
6.7,3.1,4.4,1.4,0,1,0


In [4]:
iris = pd.read_csv("iris.csv")
iris.head()

,septal_length,septal_width,petal_length,petal_width,setosa,versicolor,virginica
0,6.4,2.8,5.6,2.2,0,0,1
1,5.0,2.3,3.3,1.0,0,1,0
2,4.9,2.5,4.5,1.7,0,0,1
3,4.9,3.1,1.5,0.1,1,0,0
4,5.7,3.8,1.7,0.3,1,0,0


In [5]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   septal_length  120 non-null    float64
 1   septal_width   120 non-null    float64
 2   petal_length   120 non-null    float64
 3   petal_width    120 non-null    float64
 4   setosa         120 non-null    int64  
 5   versicolor     120 non-null    int64  
 6   virginica      120 non-null    int64  
dtypes: float64(4), int64(3)
memory usage: 6.7 KB


In [6]:
data = iris.to_numpy()
print(data.shape) #데이터 120개, 하나의 데이터가 7개이 숫자로 이루어져 있음.
print(data[:5])

x = data[:,:4]
y = data[:,4:]
print(x.shape)
print(y.shape)
print(x[:4])
print(y[:4])

(120, 7)
[[6.4 2.8 5.6 2.2 0.  0.  1. ]
 [5.  2.3 3.3 1.  0.  1.  0. ]
 [4.9 2.5 4.5 1.7 0.  0.  1. ]
 [4.9 3.1 1.5 0.1 1.  0.  0. ]
 [5.7 3.8 1.7 0.3 1.  0.  0. ]]
(120, 4)
(120, 3)
[[6.4 2.8 5.6 2.2]
 [5.  2.3 3.3 1. ]
 [4.9 2.5 4.5 1.7]
 [4.9 3.1 1.5 0.1]]
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]]


In [7]:
x = data[:,:4] #입력데이터
y = data[:,4:] #출력 데티터
#예를들어 입력이, [6.4 2.8 5.6 2.2] 이면, 출력은 [0. 0. 1.] 여야함.

#슬라이스 기준.
split_index = 100

train_x, test_x = x[:split_index], x[split_index:] #0~99, 100~199
train_y, test_y = y[:split_index], y[split_index:] #0~99, 100~199

In [8]:
model = keras.Sequential()
model.add(Input(4)) #입력 노드 4개. 입력의 feature 4개를 받기 위해서.
model.add(Dense(10, activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(3, activation='softmax')) #출력층(카테고리계수) 3개. 분류 문제에서는 softmax 사용함.

In [9]:
#model.compile(optimizer='adam', loss='crossentropy', metrics=['accuracy'])
# 보통은 model.compile(optimizer='adam', loss='crossentropy') 이렇게씀.
#crossentropy는 [0.1,0.89,0.01]이 출력이 나오면, 어떤 값이든 간에 이중에서 값이 큰지가 중요함.
#모양세 분포가 균일하게 해주는 것이 crossentorpy
#metrics accuracy 맞춘게 몇개야. 이것도 이걸 계산하기 위해 1 eopochs 한번씩 계산하려면 , 지나가고 예측하고 맞춘거 세고 전체 계수 나누고 등등 다 코딩
# 근데, categorical_을 붙이면 케라스가 대신해줌. crossentropy는 그냥 딥러닝의 이론적인거. categorical_은 케라스의 편익.
# categorical_ 이걸 붙이면, 이것은 분류 문제야. 그래서 매트릭을 accuracy라고 하면 내가 자동으로 계산해줄게가 됨.

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
#학습
model.fit(train_x, train_y, epochs=200, verbose=1)

Epoch 1/200
4/4 [==============================] - 0s 4ms/step - loss: 0.1264 - accuracy: 0.9700
Epoch 2/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1242 - accuracy: 0.9700
Epoch 3/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1236 - accuracy: 0.9700
Epoch 4/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1231 - accuracy: 0.9700
Epoch 5/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1227 - accuracy: 0.9700
Epoch 6/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1217 - accuracy: 0.9700
Epoch 7/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1210 - accuracy: 0.9700
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1206 - accuracy: 0.9700
Epoch 9/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1219 - accuracy: 0.9700
Epoch 10/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1241 - accuracy: 0.9700
Epoch 11/200
4/4 [===========

In [13]:
#모델 평가 (test set으로 함.)
loss, acc = model.evaluate(test_x, test_y)

print("loss :", loss)
print("acc :", acc)


1/1 [==============================] - 0s 124ms/step - loss: 0.0183 - accuracy: 1.0000
loss : 0.018323134630918503
acc : 1.0


In [15]:
#테스트셋을 모델에 적용하고, 저장.
#예측
y_ = model.predict(test_x)
print(y_) #출력 노드 3개라서 한데이터에 3새 숫자 들어 있음.
print(np.argmax(y_, axis=1))
#[9.90303218e-01 9.61305015e-03 8.36996696e-05] 0 정답
#[8.41818284e-03 9.90557790e-01 1.02398230e-03] 1 정답
#[1.24126411e-04 2.58139893e-03 9.97294486e-01] 2 정답

[[9.90303218e-01 9.61305015e-03 8.36996696e-05]
 [8.41818284e-03 9.90557790e-01 1.02398230e-03]
 [9.75562572e-01 2.43047792e-02 1.32663947e-04]
 [5.03746327e-03 9.94049668e-01 9.12865624e-04]
 [9.91478026e-01 8.43884889e-03 8.30660647e-05]
 [9.92868483e-01 7.05708191e-03 7.44903082e-05]
 [9.93718028e-01 6.21033227e-03 7.17181611e-05]
 [9.93978381e-01 5.94987487e-03 7.17007060e-05]
 [7.97620043e-03 9.91264105e-01 7.59696937e-04]
 [9.89297092e-01 1.06160427e-02 8.69647847e-05]
 [1.87576190e-03 1.18911505e-01 8.79212677e-01]
 [6.79519167e-03 9.88010824e-01 5.19397575e-03]
 [9.87095475e-01 1.28126275e-02 9.19151425e-05]
 [1.24126411e-04 2.58139893e-03 9.97294486e-01]
 [9.91536021e-01 8.37644748e-03 8.74481775e-05]
 [8.14807229e-03 9.32723343e-01 5.91285937e-02]
 [6.44301809e-03 9.91588533e-01 1.96852442e-03]
 [9.90263820e-01 9.64943040e-03 8.67459530e-05]
 [9.90749657e-01 9.16916039e-03 8.11655627e-05]
 [5.64416638e-03 9.93251145e-01 1.10471062e-03]]
[0 1 0 1 0 0 0 0 1 0 2 1 0 2 0 1 1 0 0 

# loss categorical_crossentropy

2가지 crossentropy 사용 방법
- categorical_crossentropy
- sparse_categorical_crossentropy

## categorical_crossentropy
y의 값이 one hot encoding인 경우
```
1,0,0
0,1,0
0,0,1
```

출력 레이어 설정
```
model.add(Dense(3, activation="softmax")) # 출력 레이어
```

loss 설정
```
model.compile(..., loss='categorical_crossentropy')
```


## sparse_categorical_crossentropy
y의 값이 one hot encoding인 경우
```
0
1
2
```

출력 레이어 설정
```
model.add(Dense(3, activation="softmax")) # 출력 레이어. 1이 아니라 클래스 수 3
```

loss 설정

category index를 준 것이다라는 의미로 spares_를 붙여줘야함.
```
model.compile(..., loss='sparse_categorical_crossentropy')
```


# iris_dnn with category index

아래의 코드는 dnn_iris_and_optimizer.ipynb의 코드를 기반으로 한다.


In [16]:
!wget https://raw.githubusercontent.com/dhrim/cau_2022/master/material/deep_learning/iris_with_category_index.csv

--2022-01-19 08:21:50--  https://raw.githubusercontent.com/dhrim/cau_2022/master/material/deep_learning/iris_with_category_index.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2218 (2.2K) [text/plain]
Saving to: ‘iris_with_category_index.csv’

iris_with_category_ 100%[===================>]   2.17K  --.-KB/s    in 0s      

2022-01-19 08:21:50 (28.0 MB/s) - ‘iris_with_category_index.csv’ saved [2218/2218]



In [19]:
!ls -al
!head iris_with_category_index.csv

total 24
drwxr-xr-x 1 root root 4096 Jan 19 08:21 .
drwxr-xr-x 1 root root 4096 Jan 19 07:39 ..
drwxr-xr-x 4 root root 4096 Jan  7 14:33 .config
-rw-r--r-- 1 root root 2720 Jan 19 07:43 iris.csv
-rw-r--r-- 1 root root 2218 Jan 19 08:21 iris_with_category_index.csv
drwxr-xr-x 1 root root 4096 Jan  7 14:33 sample_data
septal_length,septal_width,petal_length,petal_width,class
6.4,2.8,5.6,2.2,2
5.0,2.3,3.3,1.0,1
4.9,2.5,4.5,1.7,2
4.9,3.1,1.5,0.1,0
5.7,3.8,1.7,0.3,0
4.4,3.2,1.3,0.2,0
5.4,3.4,1.5,0.4,0
6.9,3.1,5.1,2.3,2
6.7,3.1,4.4,1.4,1


In [20]:
iris = pd.read_csv("iris_with_category_index.csv")
iris.head()

,septal_length,septal_width,petal_length,petal_width,class
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [21]:
data = iris.to_numpy()
print(data.shape)
print(data[:5])

(120, 5)
[[6.4 2.8 5.6 2.2 2. ]
 [5.  2.3 3.3 1.  1. ]
 [4.9 2.5 4.5 1.7 2. ]
 [4.9 3.1 1.5 0.1 0. ]
 [5.7 3.8 1.7 0.3 0. ]]


In [22]:
x = data[:,:4]
y = data[:,4:]

split_index = 100

train_x, test_x = x[:split_index], x[split_index:]
train_y, test_y = y[:split_index], y[split_index:]

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(100, 4)
(100, 1)
(20, 4)
(20, 1)


In [23]:
model = keras.Sequential()
model.add(Input(4))
model.add(Dense(10, activation='tanh'))
model.add(Dense(10, activation='tanh'))
model.add(Dense(3, activation='softmax')) # 1이 아니고 클래스 수(카테고리수) 3이다


# model.compile(optimizer="SGD", loss="categorical_crossentropy", metrics=["accuracy"])
#category index를 준 것이다라는 의미로 spares_를 붙여줘야함. 
#그러면 위에서 y 값이 0,1,2로 한개인데 클래수인 3으로 입력한것을 제대로 인식해 원핫 인코딩으로 바꿔 처리해줌
model.compile(optimizer="SGD", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

model.fit(train_x, train_y, epochs=1000, verbose=0, batch_size=20)

loss, acc = model.evaluate(test_x, test_y)
print("loss=", loss)
print("acc=", acc)
              


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 10)                50        
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                                                 
 dense_5 (Dense)             (None, 3)                 33        
                                                                 
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 101ms/step - loss: 0.0273 - accuracy: 1.0000
loss= 0.02729225531220436
acc= 1.0


In [24]:
y_ = model.predict(test_x)
print(y_)
print(np.argmax(y_, axis=1))

[[9.9411017e-01 5.4973005e-03 3.9243660e-04]
 [4.5051039e-03 9.9409109e-01 1.4038230e-03]
 [9.9053872e-01 9.0698684e-03 3.9148587e-04]
 [2.7342893e-03 9.9592030e-01 1.3453517e-03]
 [9.9383932e-01 5.7354402e-03 4.2524139e-04]
 [9.9528617e-01 4.3186136e-03 3.9523427e-04]
 [9.9545634e-01 4.1351449e-03 4.0861309e-04]
 [9.9538851e-01 4.1996939e-03 4.1174056e-04]
 [5.0092721e-03 9.9374181e-01 1.2490088e-03]
 [9.9242193e-01 7.1442476e-03 4.3383599e-04]
 [1.7311724e-03 2.7518025e-01 7.2308862e-01]
 [2.9889464e-03 9.9241322e-01 4.5977556e-03]
 [9.9257350e-01 7.0156180e-03 4.1080924e-04]
 [5.8304373e-04 5.9236631e-02 9.4018036e-01]
 [9.9389547e-01 5.6756749e-03 4.2888211e-04]
 [2.9834497e-03 9.4550526e-01 5.1511336e-02]
 [2.8376912e-03 9.9490064e-01 2.2617318e-03]
 [9.9260581e-01 6.9653755e-03 4.2878214e-04]
 [9.9403560e-01 5.5671977e-03 3.9722931e-04]
 [3.3248297e-03 9.9521232e-01 1.4628436e-03]]
[0 1 0 1 0 0 0 0 1 0 2 1 0 2 0 1 1 0 0 1]


출력 노드에는, 각 카테고리에 해당되는 깃발든 애들이 한명씩 있어서 이거는 너 깃발들어(1) 나머지 깃발들지마(0) -> 원핫 코딩 이런 데이터 셋을 가지고 얘네들을 학습시킴.

나중에, 출력된거 가지고 제일 높이든 애가 누구야?를가지고 아 이건 어느 카테고리네


---
이런 입력에 대해 이런것을 출력해봐라는 전형적인 입출력 관계를 학습에 벗어나지 않음. (이런식으로 원핫 인코딩에서 사용)